In [6]:
####################### IMPORTING ALL LIBRARIES #############################
# Core
import os
import json
import random
import itertools

# Math & Analysis
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

# Image Processing
from PIL import Image

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Vision
import torchvision.models as models
import torchvision.transforms as transforms

# Machine Learning
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_distances

# Similarity Search
import faiss

##########################################################################
# Setting working directory
os.chdir("/home/ec2-user/SageMaker/spring-2025-final-project-project-group-4")
print("Current working directory:", os.getcwd())

Current working directory: /home/ec2-user/SageMaker/spring-2025-final-project-project-group-4


In [12]:
# ------------------------------
# Load Models
# ------------------------------

# === Feature model architecture (with final FC replaced by Identity for embedding extraction) ===
class FeatureModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.base = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        self.base.fc = nn.Linear(self.base.fc.in_features, 4)  # 4 classes

    def forward(self, x):
        return self.base(x)

# === Instantiate and load feature model ===
feature_model = FeatureModel()
feature_model.load_state_dict(torch.load("Parsa/checkpoint/feature_model.pth", map_location=torch.device("cpu")))
feature_model.eval()

# Extract features (remove final classification layer)
feature_extractor = nn.Sequential(*list(feature_model.base.children())[:-1])  # Output: [batch, 512, 1, 1]

# === Siamese model architecture ===
class SiameseNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128)
        )

    def forward(self, input1, input2):
        output1 = self.fc(input1)
        output2 = self.fc(input2)
        return output1, output2

# === Instantiate and load Siamese model ===
siamese_model = SiameseNetwork()
siamese_model.load_state_dict(torch.load("Parsa/checkpoint/siamese_model.pth", map_location=torch.device("cpu")))
siamese_model.eval()

# ------------------------------
# Image Preprocessing
# ------------------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

def load_image_embedding(img_path):
    img = Image.open(img_path).convert("RGB")
    img_tensor = transform(img).unsqueeze(0)
    with torch.no_grad():
        features = feature_extractor(img_tensor).squeeze().numpy()
    return features

# ------------------------------
# Load Metadata
# ------------------------------
with open("Parsa/parsa's_wardrobe/metadata.json") as f:
    metadata = json.load(f)

wardrobe_items = metadata["shirts"] + metadata.get("pants", []) + metadata.get("shorts", []) + metadata.get("t-shirts", [])

# ------------------------------
# Generate Embeddings
# ------------------------------
embeddings = []
paths = []
folder_map = {
    "shirt": "shirts",
    "t-shirt": "t-shirts",
    "short": "shorts",
    "pant": "pants",  # In case any metadata used singular form
    "pants": "pants",
    "shorts": "shorts",
    "shirts": "shirts",
    "t-shirts": "t-shirts"
}

for item in wardrobe_items:
    subfolder = folder_map.get(item["category"], item["category"])
    filepath = os.path.join("Parsa/parsa's_wardrobe", subfolder, item["filename"])
    emb = load_image_embedding(filepath)
    embeddings.append(emb)
    paths.append(item)

embeddings = np.vstack(embeddings)

# Save embeddings (optional)
with open("Parsa/testing_code/clothing_embeddings.json", "w") as f:
    json.dump([{**paths[i], "embedding": embeddings[i].tolist()} for i in range(len(paths))], f)

# ------------------------------
# Build FAISS Index
# ------------------------------
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

faiss.write_index(index, "Parsa/testing_code/clothing_faiss.index")

# ------------------------------
# Outfit Matching Logic
# ------------------------------
def is_compatible_pair(item1, item2):
    if item1["category"] in ["shirt", "t-shirt"] and item2["category"] in ["pants", "shorts"]:
        return True
    if item2["category"] in ["shirt", "t-shirt"] and item1["category"] in ["pants", "shorts"]:
        return True
    return False

def get_top_outfits(top_k=5):
    outfit_scores = []
    for i in range(len(paths)):
        for j in range(i+1, len(paths)):
            if is_compatible_pair(paths[i], paths[j]):
                emb1 = torch.tensor(embeddings[i]).unsqueeze(0)
                emb2 = torch.tensor(embeddings[j]).unsqueeze(0)
                with torch.no_grad():
                    output1, output2 = siamese_model(emb1, emb2)
                    score = torch.nn.functional.pairwise_distance(output1, output2).item()
                outfit_scores.append(((paths[i], paths[j]), score))

    outfit_scores.sort(key=lambda x: x[1], reverse=True)
    return outfit_scores[:top_k]

# ------------------------------
# Show Recommendations
# ------------------------------
top_outfits = get_top_outfits(top_k=5)

for idx, ((item1, item2), score) in enumerate(top_outfits):
    print(f"\nOutfit {idx+1} — Score: {score:.3f}")
    print(f" Top: {item1['filename']} ({item1['category']}, {item1['color']}, {item1['style']})")
    print(f" Bottom: {item2['filename']} ({item2['category']}, {item2['color']}, {item2['style']})")


Outfit 1 — Score: 0.587
 Top: 24_shirts.jpg (shirt, red, casual)
 Bottom: 02_shorts.jpg (shorts, white, ['sporty'])

Outfit 2 — Score: 0.537
 Top: 24_shirts.jpg (shirt, red, casual)
 Bottom: 11_pants.jpg (pants, white, ['casual'])

Outfit 3 — Score: 0.529
 Top: 02_shorts.jpg (shorts, white, ['sporty'])
 Bottom: 05_t-shirts.jpg (t-shirt, black, sporty)

Outfit 4 — Score: 0.527
 Top: 27_shirts.jpg (shirt, gray, casual)
 Bottom: 02_shorts.jpg (shorts, white, ['sporty'])

Outfit 5 — Score: 0.517
 Top: 02_shorts.jpg (shorts, white, ['sporty'])
 Bottom: 02_t-shirts.jpg (t-shirt, white, sporty)
